# Importing the libraries

In [1]:
from platform import python_version

print(python_version())

3.8.2


In [1]:
!pip install -q tensorflow

import numpy as np
import tensorflow as tf
import random as rn

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(37)
rn.seed(1254)
tf.random.set_seed(89)

In [3]:
from tensorflow.python.keras import backend as K

# force tensorflow to use a single thread
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=config)
K.set_session(sess)

In [4]:
## Imports libs
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

from tensorflow.keras import layers
from keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

Segmentation Models: using `tf.keras` framework.


Using TensorFlow backend.


In [5]:
!pip install -U -q segmentation-models
!pip install -q keras
!pip install -q tensorflow-estimator

import pandas as pd
import cv2
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle
from tqdm import tqdm
import pandas as pd
from sklearn.utils import class_weight

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.metrics import confusion_matrix

In [6]:
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

# Data preparation 

In [7]:
train_datagen = ImageDataGenerator(horizontal_flip = True)
training_set = train_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\train_files_cropped',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

Found 2024 images belonging to 7 classes.


In [8]:
test_datagen = ImageDataGenerator()
test_set = test_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\test_files_cropped',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 507 images belonging to 7 classes.


# Downloading the pre-trained model

In [9]:
# Downloading the pre-trained model
model = load_model(r'C:\Users\HP\FER_SL\models\affectnet_emotions\mobilenet_7.h5')

In [10]:
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [12]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

In [13]:
params = count_params(model)
assert params['non_trainable_params'] == 21888
assert params['trainable_params'] == 3471175

# Removing the last prediction layer 

In [14]:
new_model = Sequential()
for layer in model.layers[:-1]:
    new_model.add(layer)

# Freezing the previous layers 

In [15]:
for layer in new_model.layers:
    layer.trainable = False

# Adding a new dense layer

In [16]:
new_model.add(Dense(units=7, input_shape=(224,224), activation='softmax'))

In [17]:
#new_model.add(tf.keras.layers.Flatten())

In [18]:
#new_model.build(input_shape)
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

# Training only the last layer 

In [19]:
params = count_params(new_model)
assert params['non_trainable_params'] == 3491264
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 1799

In [20]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
net_description = 'FT_HF_mobileNet'
nb_train_samples = 2024
nb_validation_samples = 507
mc = ModelCheckpoint(net_description+'.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_accuracy',patience=2)

In [22]:
hist1=new_model.fit(x=training_set, steps_per_epoch=nb_train_samples//BATCH_SIZE, initial_epoch=0, callbacks=[mc, es], validation_data = test_set, validation_steps=nb_validation_samples // BATCH_SIZE, epochs=3, verbose=1)

Epoch 1/3
25/25 [==============================] - 28s 989ms/step - loss: 2.3290 - accuracy: 0.1005 - val_loss: 2.0648 - val_accuracy: 0.0573

Epoch 00001: val_accuracy improved from -inf to 0.05729, saving model to FT_HF_mobileNet.h5
Epoch 2/3
25/25 [==============================] - 27s 1s/step - loss: 1.8202 - accuracy: 0.2900 - val_loss: 1.8979 - val_accuracy: 0.0781

Epoch 00002: val_accuracy improved from 0.05729 to 0.07812, saving model to FT_HF_mobileNet.h5
Epoch 3/3
25/25 [==============================] - 21s 836ms/step - loss: 1.6445 - accuracy: 0.3900 - val_loss: 1.6370 - val_accuracy: 0.2552

Epoch 00003: val_accuracy improved from 0.07812 to 0.25521, saving model to FT_HF_mobileNet.h5


# Unfreezing the previous layers

In [23]:
for layer in new_model.layers:
    layer.trainable = True

In [24]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

In [25]:
params = count_params(new_model)
assert params['non_trainable_params'] == 21888
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 3471175 

# Retraining the model with all layers

In [26]:
start_epoch=len(hist1.history['loss'])
new_model.load_weights(net_description+'.h5')
new_model.compile(optimizer=Adam(learning_rate=1e-4,decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
SECOND_EPOCHS=start_epoch+5
mc = ModelCheckpoint(net_description+'_ft.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
#es=EarlyStopping(monitor='val_accuracy',patience=2 )
print("All Tuning")

All Tuning


In [28]:
hist2=new_model.fit(x=training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, initial_epoch=start_epoch, validation_data = test_set, validation_steps=test_set.samples // BATCH_SIZE,callbacks=[mc])

Epoch 4/8
25/25 [==============================] - 103s 4s/step - loss: 1.5260 - accuracy: 0.4538 - val_loss: 1.6576 - val_accuracy: 0.2604

Epoch 00004: val_accuracy improved from -inf to 0.26042, saving model to FT_HF_mobileNet_ft.h5
Epoch 5/8
25/25 [==============================] - 101s 4s/step - loss: 1.2406 - accuracy: 0.5638 - val_loss: 1.3839 - val_accuracy: 0.4062

Epoch 00005: val_accuracy improved from 0.26042 to 0.40625, saving model to FT_HF_mobileNet_ft.h5
Epoch 6/8
25/25 [==============================] - 111s 4s/step - loss: 1.1678 - accuracy: 0.5799 - val_loss: 1.2452 - val_accuracy: 0.5104

Epoch 00006: val_accuracy improved from 0.40625 to 0.51042, saving model to FT_HF_mobileNet_ft.h5
Epoch 7/8
25/25 [==============================] - 111s 4s/step - loss: 0.9928 - accuracy: 0.6388 - val_loss: 1.1098 - val_accuracy: 0.5625

Epoch 00007: val_accuracy improved from 0.51042 to 0.56250, saving model to FT_HF_mobileNet_ft.h5
Epoch 8/8
25/25 [==============================

In [29]:
new_model.load_weights(net_description+'_ft.h5')

In [30]:
new_model.compile(optimizer=SGD(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
#new_model.summary()
mc = ModelCheckpoint(net_description+'_ft_sgd.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
second_epoch=start_epoch+len(hist2.history['loss'])
THIRD_EPOCHS=2+second_epoch

print('ALL SGD tuning')
hist3=new_model.fit(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=THIRD_EPOCHS, verbose=1, 
                    initial_epoch=second_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE,callbacks=[mc])

ALL SGD tuning
Epoch 9/10
25/25 [==============================] - 121s 5s/step - loss: 0.8568 - accuracy: 0.7038 - val_loss: 0.9895 - val_accuracy: 0.6354

Epoch 00009: val_accuracy improved from -inf to 0.63542, saving model to FT_HF_mobileNet_ft_sgd.h5
Epoch 10/10
25/25 [==============================] - 108s 4s/step - loss: 0.7722 - accuracy: 0.7281 - val_loss: 0.9894 - val_accuracy: 0.6458

Epoch 00010: val_accuracy improved from 0.63542 to 0.64583, saving model to FT_HF_mobileNet_ft_sgd.h5


# Evaluation

In [31]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [32]:
USE_PREPROCESSING=False
INPUT_SIZE = (224,224)
import numpy as np
y_val=np.array(test_set.labels)
y_scores_val=[]
#new_model = load_model(r'C:\Users\HP\SLR\src\FT_HF_mobileNet_ft_sgd.h5')
for filepath in log_progress(test_set.filepaths):
    img=cv2.imread(filepath)
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,INPUT_SIZE)
    
    test_image = np.asarray(img)
    test_image = np.expand_dims(test_image, axis = 0)

    scores=new_model.predict(test_image)

    #print(scores)
    y_scores_val.append(scores)

y_scores_val=np.concatenate(y_scores_val, axis=0)
#print(y_scores_val.shape,y_val.shape)

In [33]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}

y_pred=np.argmax(y_scores_val,axis=1)
acc=100.0*(y_val==y_pred).sum()/len(y_val)
print('Accuracy:',acc)

y_train=np.array(training_set.labels)

for i in range(y_scores_val.shape[1]):
    _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
    print('%s %d/%d acc: %f' %(idx_to_class[i],(y_train==i).sum(),(y_val==i).sum(),100*_val_acc))

Accuracy: 64.69428007889546
Anger 371/132 acc: 77.272727
Disgust 156/31 acc: 51.612903
Fear 251/57 acc: 50.877193
Happiness 160/19 acc: 63.157895
Neutral 153/42 acc: 28.571429
Sadness 290/59 acc: 55.932203
Surprise 643/167 acc: 74.251497


In [34]:
from sklearn.metrics import classification_report

target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Anger       0.78      0.77      0.78       132
     Disgust       0.53      0.52      0.52        31
        Fear       0.54      0.51      0.52        57
       Happy       0.39      0.63      0.48        19
     Neutral       0.71      0.29      0.41        42
     Sadness       0.41      0.56      0.47        59
    Surprise       0.76      0.74      0.75       167

    accuracy                           0.65       507
   macro avg       0.59      0.57      0.56       507
weighted avg       0.67      0.65      0.65       507

